<a href="https://colab.research.google.com/github/xiaofanpaiooo-code/whisper-lora/blob/xiaofanpaiooo-code-patch-1/whisper_zimu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==========================================
# 部署说明：
# 1. 复制此代码到 Colab。
# 2. 运行。
# 3. 英文专用版：听英文 -> 写英文 (速度极快)。
# ==========================================

import os
import sys
import subprocess
import time

# 1. 环境安装
def install_dependencies():
    packages = ["faster-whisper", "fastapi", "uvicorn", "python-multipart", "pyngrok"]
    # -q 表示静默安装
    subprocess.check_call([sys.executable, "-m", "pip", "install"] + packages)

try:
    import faster_whisper
    import fastapi
    from pyngrok import ngrok
    import uvicorn
except ImportError:
    print("正在安装依赖环境，请稍候...", flush=True)
    install_dependencies()
    print("环境安装完成！", flush=True)

# ================= 配置区域 =================
NGROK_AUTH_TOKEN = "37hXaj4l3VhFmrso4FTbEWYE3Li_7Z3vCLhdMAPb2BpiVufAw"
# ===========================================

# 2. 生成独立服务脚本
server_code = f"""
import os
import sys
import shutil
import time
import subprocess
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from faster_whisper import WhisperModel
from pyngrok import ngrok
import uvicorn
import torch

def log(msg):
    print(msg, flush=True)

def kill_port(port):
    try:
        command = f"fuser -k {{port}}/tcp"
        subprocess.run(command, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        time.sleep(1)
    except:
        pass

NGROK_AUTH_TOKEN = "{NGROK_AUTH_TOKEN}"

kill_port(8000)
app = FastAPI()

if not torch.cuda.is_available():
    log("\\n❌ 严重错误：未检测到 GPU！请切换运行时类型为 T4 GPU。\\n")
    sys.exit(1)

log("正在加载 Whisper 模型 (Tiny + float16)...")
try:
    # 英文场景下 tiny 模型效果惊人地好，且速度最快
    model = WhisperModel("tiny", device="cuda", compute_type="float16")
    log("✅ 模型加载完毕！")
except Exception as e:
    log(f"❌ 模型加载失败: {{e}}")
    sys.exit(1)

@app.post("/transcribe")
async def transcribe(file: UploadFile = File(...)):
    start_time = time.time()
    temp_filename = f"temp_{{file.filename}}"
    try:
        with open(temp_filename, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)

        # ========================================================
        # 英文专用配置
        # ========================================================
        segments, info = model.transcribe(
            temp_filename,
            beam_size=1,
            best_of=1,
            language="en",     # <--- 关键修改：指定为英语

            # 1. 解决复读问题
            condition_on_previous_text=False,

            # 2. 英文引导提示 (防止开头幻觉)
            initial_prompt="Hello, welcome to the stream. This is an English transcript.",

            # 3. 提速优化
            temperature=0.0,
            vad_filter=True,
            vad_parameters=dict(min_silence_duration_ms=500)
        )

        text_result = "".join([segment.text for segment in segments])

        process_time = time.time() - start_time
        log(f"耗时: {{process_time:.2f}}s | 结果: {{text_result}}")
        return JSONResponse(content={{"text": text_result.strip()}})
    except Exception as e:
        log(f"❌ 推理出错: {{e}}")
        return JSONResponse(content={{"error": str(e)}}, status_code=500)
    finally:
        if os.path.exists(temp_filename):
            os.remove(temp_filename)

if __name__ == "__main__":
    if not NGROK_AUTH_TOKEN:
        log("❌ 错误：Token 未设置")
    else:
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        try:
            public_url = ngrok.connect(8000).public_url
            log(f"\\n======== 英文转写服务已启动 ========")
            log(f"API 接口地址: {{public_url}}/transcribe")
            log(f"==================================\\n")
            uvicorn.run(app, host="0.0.0.0", port=8000)
        except Exception as e:
            log(f"❌ 启动失败: {{e}}")
"""

with open("server_main.py", "w", encoding="utf-8") as f:
    f.write(server_code)

print("已生成英文版脚本，正在启动...", flush=True)

env = os.environ.copy()
env["PYTHONUNBUFFERED"] = "1"

process = subprocess.Popen(
    [sys.executable, "server_main.py"],
    env=env,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

try:
    for line in process.stdout:
        print(line, end="")
except KeyboardInterrupt:
    print("服务已停止")
    process.terminate()

正在安装依赖环境，请稍候...
环境安装完成！
已生成英文版脚本，正在启动...
正在加载 Whisper 模型 (Tiny + float16)...
✅ 模型加载完毕！
                                                                                                    
Installing ngrok ... 
                                                                                                    
======== 英文转写服务已启动 ========
API 接口地址: https://unavoidable-amber-plinthlike.ngrok-free.dev/transcribe

INFO:     Started server process [831]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
耗时: 1.16s | 结果:  A lot of people starting out in their taro journey don't like to read river source. They like to have all the cards upright and to read like that because I guess it's a little bit less.
INFO:     64.181.228.168:0 - "POST /transcribe HTTP/1.1" 200 OK
耗时: 0.25s | 结果:  Scary, but actually reading with the vessels makes the tower own a lot richer. Not only why the tower has 78 